# Clot Burden Prediction

We perform a univariable regression for each target variable and explanatory variable. We then perform multivariable regressions. For these, we focus on body composition only, cardiopulmonary features only, and then a composite model. For each of these, we perform three forms of feature selection, using (1) recursive feature elimination with cross validation, (2) forward sequential feature selection with cross validation, and (3) backward feature selection with cross validation. For these groups of selected features, we also perform sensitivities controlling for gender, age, and both gender and age.

# Imports

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import pickle
from tqdm.notebook import trange, tqdm

# Custom / Lcoal
from config import model_config
from regression import reg

# Stats
from scipy.stats import shapiro
from sklearn.feature_selection import(
	RFECV, SequentialFeatureSelector,
)
from sklearn.metrics import (
    make_scorer, r2_score
)
from sklearn.linear_model import (
	LinearRegression, LogisticRegression, LassoCV
)
from sklearn.model_selection import (
	train_test_split, cross_val_score,
    RepeatedKFold
)
from sklearn.pipeline import (
	Pipeline
)
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Global vars
SEED = 123
TEST_SIZE = 0.25
HEATMAP_COLORS = sns.diverging_palette(h_neg=250, h_pos=359, as_cmap=True)
SIGNIFICANCE_CUTOFF = 0.05
CV_FOLDS = 5
CUSTOM_CV = RepeatedKFold(n_splits=CV_FOLDS, n_repeats=20, random_state=SEED)


# Cov type
# Options:
#  - robust
#  - clustered
COV_TYPE = 'robust'


# Import/Preprocess Data

In [33]:
with open(Path('../data/prediction_data.pkl'), 'rb') as f:
	data = pickle.load(f)
	
X = data.get('X')
y = data.get('y')
body_features = data.get('body_features')
cardio_features = data.get('cardio_features')
control_features = data.get('controls')
all_features = body_features + cardio_features + control_features

print(X.shape)
print(y.shape)
print(body_features)
print(cardio_features)
print(control_features)

(96, 37)
(96, 7)
['volume_visceral_fat', 'density_visceral_fat', 'mass_visceral_fat', 'volume_subcutaneous_fat', 'density_subcutaneous_fat', 'mass_subcutaneous_fat', 'volume_intermuscular_fat', 'density_intermuscular_fat', 'mass_intermuscular_fat', 'volume_muscle', 'density_muscle', 'mass_muscle', 'volume_bone', 'density_bone', 'mass_bone', 'bmi', 'bsa']
['emphysema_volume_950hu', 'lung_volume', 'extrapulmonary_artery_volume', 'extrapulmonary_vein_volume', 'intrapulmonary_artery_volume', 'intrapulmonary_vein_volume', 'artery_vein_ratio', 'bv5', 'bv10', 'pb_larger_10', 'pv_diameter', 'a_diameter', 'pv_a', 'heart_volume', 'airway_volume', 'airway_ratio', 'ild_volume', 'ild_ratio']
['age', 'gender_cl_Male']


In [34]:
pe_numbers = y.index.str[:-2]
pe_numbers

Index(['PE1', 'PE12', 'PE12', 'PE14', 'PE15', 'PE15', 'PE15', 'PE15', 'PE16',
       'PE16', 'PE17', 'PE18', 'PE18', 'PE18', 'PE18', 'PE19', 'PE2', 'PE20',
       'PE21', 'PE22', 'PE22', 'PE22', 'PE23', 'PE23', 'PE23', 'PE24', 'PE24',
       'PE25', 'PE25', 'PE25', 'PE27', 'PE27', 'PE28', 'PE3', 'PE3', 'PE3',
       'PE31', 'PE32', 'PE32', 'PE32', 'PE32', 'PE33', 'PE34', 'PE34', 'PE35',
       'PE36', 'PE36', 'PE37', 'PE37', 'PE37', 'PE4', 'PE40', 'PE40', 'PE41',
       'PE41', 'PE41', 'PE41', 'PE42', 'PE42', 'PE43', 'PE44', 'PE45', 'PE47',
       'PE48', 'PE48', 'PE49', 'PE5', 'PE5', 'PE51', 'PE51', 'PE51', 'PE52',
       'PE52', 'PE52', 'PE52', 'PE52', 'PE52', 'PE52', 'PE53', 'PE54', 'PE54',
       'PE54', 'PE56', 'PE56', 'PE6', 'PE6', 'PE6', 'PE6', 'PE6', 'PE6', 'PE6',
       'PE7', 'PE8', 'PE8', 'PE8', 'PE9'],
      dtype='object')

# OLS Regression Functions

In [35]:
def get_params(model, X, y):
    """Returns pd.Series of coefs for comparison with statsmodels params."""
    model.fit(X, y)
    coef = pd.Series(model.coef_, index=model.feature_names_in_)
    coef['const'] = model.intercept_
    return coef.sort_values()

In [36]:
def model_residual_correlation(model):
    """Returns measure of correlation."""
    return np.corrcoef(np.arange(len(model.resid)), model.resid)[1, 0]

In [37]:
def fit_model(X, y):
    """Fit statsmodels OLS model with robust SEs and sklearn OLS model."""
    # Fit statsmodels model for pvalues and coef
    if COV_TYPE == 'robust':
        model_sm = sm.OLS(y, X).fit(cov_type='HC3')
    elif COV_TYPE == 'clustered':
        model_sm = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': pe_numbers})
    # Define sklearn model for CV evaluation
    model_sk = LinearRegression(fit_intercept=True, n_jobs=-1)
    # Check that model params match
    sk_model_params = get_params(model_sk, X, y)
    sm_model_params = model_sm.params.sort_values()
    params_match = np.isclose(sk_model_params, sm_model_params, atol=1e-5)
    if not np.all(params_match):
        print(f"Regressions on {y.name} did not match for sklearn and statsmodels. CV scores may differ.")
    return model_sm, model_sk

In [38]:
def store_model_results(model_sm, model_sk, X, y):
    """
    Params:
        - model_sm: statsmodel model for coefs, pvalues, and residuals.
        - model_sk: sklearn model for cross validation
        - X: X data.
        - y: y data.
    """
    # Calculate CV scores
    cv_scores = cross_val_score(
        model_sk, X, y, 
        scoring='r2', 
        cv=CUSTOM_CV, n_jobs=-1
    )
    # Store model results
    model_results = pd.DataFrame(
        {
            'y': y.name,
            'model_dfn': [tuple(X.columns.values)],
            'nobs': model_sm.nobs,
            'shapiro_resid_pvalue': shapiro(model_sm.resid).pvalue,
            'metric_train': model_sk.score(X, y),
            'metric_cv_mean': np.mean(np.maximum(cv_scores, np.zeros_like(cv_scores))),
            'metric_cv_std': np.std(np.maximum(cv_scores, np.zeros_like(cv_scores))),
            'fpvalue': model_sm.f_pvalue
        }
    )
    # Set model index
    model_results = model_results.set_index(['y', 'model_dfn'])
    return model_results

In [39]:
def store_coef_results(model_sm, y):
    """
    Params:
        - model_sm: statsmodel model for coefs, pvalues, and residuals.
        - y: y data.
    """
    results = pd.DataFrame(
        {
            'model_dfn': [tuple(model_sm.params.index) for _ in range(len(model_sm.params))],
            'coef': model_sm.params, 
            'pval': model_sm.pvalues,
        },
    )
    results['signif'] = results['pval'].apply(reg.add_significance)
    results = results.reset_index(names='x')
    results['y'] = y.name
    results = results.pivot(index=['y', 'model_dfn'], columns=['x'], values=['coef', 'pval', 'signif'])
    results.columns = ['_'.join(idx) for idx in results.columns]
    return results

In [40]:
def combine_model_results(model_sm, model_sk, X, y):
    model_results = store_model_results(model_sm, model_sk, X, y)
    coef_results = store_coef_results(model_sm, y)
    assert model_results.shape[0] == coef_results.shape[0] 
    combined_results = pd.concat([model_results, coef_results], axis=1)
    return combined_results

In [41]:
def backward_stepwise_selection(X, y, cutoff):
    # Make copies of X, y
    X_temp = sm.add_constant(X.copy())
    y_temp = y.copy()
    
    # Fit initial model
    if COV_TYPE == 'robust':
        model_sm = sm.OLS(y_temp, X_temp).fit(cov_type='HC3')
    elif COV_TYPE == 'clustered':
        model_sm = sm.OLS(y_temp, X_temp).fit(cov_type='cluster', cov_kwds={'groups': pe_numbers})
    coefs = model_sm.params[1:]
    pvals = model_sm.pvalues[1:]
    df_temp = pd.DataFrame({
        'coefs': coefs,
        'pvals': pvals
    })
    current_varlist = list(coefs.index.values)

    # Store progression in a list of lists
    progression = list()
    progression.append(dict(zip(coefs.index.values, zip(coefs.values, pvals.values))))
    
    # Iterate until all are stat signif
    while not np.all(df_temp['pvals'] < cutoff):
        
        # Drop the variable with the highest pvalue
        new_vars = df_temp.drop(index=df_temp['pvals'].idxmax()).index.values
        
        # If remaining varlist is empty, break and return the last regression results
        if len(new_vars) == 0:
            break

        # Subset X to new list of variables
        X_temp = sm.add_constant(X_temp.loc[:, new_vars])
        
        # Re-fit model
        model_sm = sm.OLS(y_temp, X_temp).fit(cov_type='HC3')
        coefs = model_sm.params[1:]
        pvals = model_sm.pvalues[1:]
        df_temp = pd.DataFrame({
            'coefs': coefs,
            'pvals': pvals
        })
        progression.append(dict(zip(coefs.index.values, zip(coefs.values, pvals.values))))
        current_varlist = [var for var in model_sm.params.index.values if var != 'const']
    
    return current_varlist, progression

# Example
feat_out, prog = backward_stepwise_selection(X[body_features], np.array(y['total_clot_burden']).ravel(), 0.05)
print(feat_out)

['volume_intermuscular_fat', 'mass_intermuscular_fat', 'bsa']


### Example

In [42]:
target = 'total_clot_burden'
features = 'density_visceral_fat'
X_temp = sm.add_constant(X[features])
y_temp = y[target]
model_sm, model_sk = fit_model(X_temp, y_temp)
combine_model_results(model_sm, model_sk, X_temp, y_temp)

,,nobs,shapiro_resid_pvalue,metric_train,metric_cv_mean,metric_cv_std,fpvalue,coef_const,coef_density_visceral_fat,pval_const,pval_density_visceral_fat,signif_const,signif_density_visceral_fat
y,model_dfn,,,,,,,,,,,,
total_clot_burden,"(const, density_visceral_fat)",96.0,1.260388e-08,0.059386,0.030212,0.040489,0.02186,0.0,-0.243692,1.0,0.019724,,*


# Perform univariable regressions

In [43]:
univariable_results = pd.DataFrame()

for target in tqdm(model_config.num_targets):
    for feature in all_features:
        # Fit model
        X_temp = sm.add_constant(X[feature])
        y_temp = y[target]
        model_sm, model_sk = fit_model(X_temp, y_temp)

        # Store results
        univariable_results = pd.concat(
            [univariable_results, combine_model_results(model_sm, model_sk, X_temp, y_temp)],
            axis=0
        )
        
print(univariable_results.shape)

univariable_results = univariable_results.reset_index()
univariable_results['selection_method'] = 'All'
univariable_results['model_dfn'] = univariable_results['model_dfn'].apply(lambda x: x[1])
univariable_results['category'] = 'univariable_' + univariable_results['model_dfn']
univariable_results['controls'] = 'None'
univariable_results.index = univariable_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
univariable_results.index.name = 'Lookup'

print(univariable_results.shape)

univariable_results.tail()

  0%|          | 0/7 [00:00<?, ?it/s]

(259, 120)
(259, 125)


,y,model_dfn,nobs,shapiro_resid_pvalue,metric_train,metric_cv_mean,metric_cv_std,fpvalue,coef_const,coef_volume_visceral_fat,...,signif_ild_ratio,coef_age,pval_age,signif_age,coef_gender_cl_Male,pval_gender_cl_Male,signif_gender_cl_Male,selection_method,category,controls
Lookup,,,,,,,,,,,,,,,,,,,,,
univariable_airway_ratio%All%centralartery%None,centralartery,airway_ratio,96.0,1.702099e-11,0.020135,0.009799,0.018359,0.298965,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_airway_ratio,None
univariable_ild_volume%All%centralartery%None,centralartery,ild_volume,96.0,4.270844e-13,0.001456,0.000109,0.000496,0.751051,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_ild_volume,None
univariable_ild_ratio%All%centralartery%None,centralartery,ild_ratio,96.0,4.059170e-12,0.017948,0.008001,0.015297,0.126185,0.0,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_ild_ratio,None
univariable_age%All%centralartery%None,centralartery,age,96.0,1.521341e-12,0.012366,0.003880,0.010416,0.305216,0.0,NaN,...,NaN,0.111203,0.30257,,NaN,NaN,NaN,All,univariable_age,None
univariable_gender_cl_Male%All%centralartery%None,centralartery,gender_cl_Male,96.0,3.216951e-12,0.019572,0.037272,0.052933,0.051687,-0.14673,NaN,...,NaN,NaN,NaN,NaN,0.426849,0.048746,*,All,univariable_gender_cl_Male,None


## Control for age and gender

In [44]:
univariable_control_results = pd.DataFrame()

for target in tqdm(model_config.num_targets):
    for feature in all_features:

        if feature in model_config.controls_encoded:
            continue
            
        # Fit model
        features = [feature, 'gender_cl_Male']
        X_temp = sm.add_constant(X[features])
        y_temp = y[target]
        model_sm, model_sk = fit_model(X_temp, y_temp)

        # Store results
        univariable_control_results = pd.concat(
            [univariable_control_results, combine_model_results(model_sm, model_sk, X_temp, y_temp)],
            axis=0
        )
        
print(univariable_control_results.shape)

univariable_control_results = univariable_control_results.reset_index()
univariable_control_results['selection_method'] = 'All'
univariable_control_results['model_dfn'] = univariable_control_results['model_dfn'].apply(lambda x: x[1])
univariable_control_results['category'] = 'univariable_controls_' + univariable_control_results['model_dfn']
univariable_control_results['controls'] = 'None'
univariable_control_results.index = univariable_control_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
univariable_control_results.index.name = 'Lookup'

print(univariable_control_results.shape)

univariable_control_results.tail()

  0%|          | 0/7 [00:00<?, ?it/s]

(245, 117)
(245, 122)


,y,model_dfn,nobs,shapiro_resid_pvalue,metric_train,metric_cv_mean,metric_cv_std,fpvalue,coef_const,coef_gender_cl_Male,...,signif_airway_ratio,coef_ild_volume,pval_ild_volume,signif_ild_volume,coef_ild_ratio,pval_ild_ratio,signif_ild_ratio,selection_method,category,controls
Lookup,,,,,,,,,,,,,,,,,,,,,
univariable_controls_heart_volume%All%centralartery%None,centralartery,heart_volume,96.0,3.470715e-12,0.022395,0.030928,0.047601,0.152671,-0.138433,0.402714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_controls_heart_volume,None
univariable_controls_airway_volume%All%centralartery%None,centralartery,airway_volume,96.0,4.232489e-11,0.044432,0.029638,0.053491,0.085889,-0.0848,0.24669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_controls_airway_volume,None
univariable_controls_airway_ratio%All%centralartery%None,centralartery,airway_ratio,96.0,5.575437e-11,0.039828,0.037410,0.061116,0.057399,-0.147182,0.428167,...,,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_controls_airway_ratio,None
univariable_controls_ild_volume%All%centralartery%None,centralartery,ild_volume,96.0,7.329235e-12,0.022348,0.031437,0.052612,0.078758,-0.157762,0.458944,...,NaN,-0.079798,0.476989,,NaN,NaN,NaN,All,univariable_controls_ild_volume,None
univariable_controls_ild_ratio%All%centralartery%None,centralartery,ild_ratio,96.0,2.277056e-11,0.036318,0.039297,0.061264,0.031093,-0.142485,0.4145,...,NaN,NaN,NaN,NaN,-0.12471,0.156036,,All,univariable_controls_ild_ratio,None


# Perform multivariable regressions

In [45]:
MAX_NUM_REGRESSORS = y.shape[0] // 10
print(f"MAX_NUM_REGRESSORS: {MAX_NUM_REGRESSORS}")

multivariable_results = pd.DataFrame()

for target in tqdm(model_config.num_targets):
    
    low_alpha = -3
    high_alpha = 4
    
    lassoCV = LassoCV(
        alphas=np.logspace(low_alpha, high_alpha, 100),
        cv=CUSTOM_CV,
        fit_intercept=True,
        max_iter=100_000,
        tol=0.001,
        n_jobs=-1
    )

    exceeds_max_num_regressors = True
    while exceeds_max_num_regressors:
        lassoCV.fit(X, y[target])
        coefs = pd.DataFrame(
            {'coef': lassoCV.coef_},
            index=lassoCV.feature_names_in_
        )
        remaining_features_lasso = coefs.loc[~np.isclose(coefs['coef'], 0.0), :].index.values

        if len(remaining_features_lasso) > MAX_NUM_REGRESSORS:
            low_alpha += 0.2
            lassoCV.set_params(**{'alphas': np.logspace(low_alpha, high_alpha, 50)})
        else:
            exceeds_max_num_regressors = False
            
    print(f"{target:<20s} alpha={lasso.alpha_:.3f}, feats: {remaining_features_lasso}")
    
    # Fit models
    X_temp_lasso = sm.add_constant(X[remaining_features_lasso])
    y_temp = y[target]
    model_sm_lasso, model_sk_lasso = fit_model(X_temp_lasso, y_temp)

    # Collect model/coef information and store
    model_eval = store_model_results(model_sm_lasso, model_sk_lasso, X_temp_lasso, y_temp)
    model_coefs = store_coef_results(model_sm_lasso, y_temp)
    model_results = pd.concat([model_eval, model_coefs], axis=1)
    multivariable_results = pd.concat([multivariable_results, model_results], axis=0)


MAX_NUM_REGRESSORS: 9


  0%|          | 0/7 [00:00<?, ?it/s]

NameError: name 'lasso' is not defined

In [ ]:
multivariable_results = multivariable_results.reset_index()
multivariable_results['selection_method'] = 'LassoCV'
multivariable_results['category'] = 'composite'
multivariable_results['controls'] = 'None'
multivariable_results.index = multivariable_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
multivariable_results.index.name = 'Lookup'
print(multivariable_results.shape)
multivariable_results.head()

# Combine Univariate and Multivariable regression results

In [ ]:
ols_results = pd.concat([univariable_results, multivariable_results], axis=0)
if COV_TYPE == 'robust':
    ols_results.to_csv('../output/regressions/ols_results_robust.csv')
elif COV_TYPE == 'clustered':
    ols_results.to_csv('../output/regressions/ols_results_clustered.csv')